In [ ]:
#TODO: reduce size vocabulary
#TODO: check size embedding
#TODO: tensorboard

In [21]:
import pandas as pd

df = pd.read_csv('data/isear_processed.csv',header=0, index_col=0, names=['text','sentiment'])

In [22]:
df['sentiment'].unique()

array(['joy', 'fear', 'anger', 'sadness', 'disgust', 'shame', 'guilt'],
      dtype=object)

In [23]:
df.head()

,text,sentiment
0,"During the period of falling in love, each tim...",joy
1,When I was involved in a traffic accident.,fear
2,When I was driving home after several days of...,anger
3,When I lost the person who meant the most to me.,sadness
4,The time I knocked a deer down - the sight of ...,disgust


In [24]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
sequences = pad_sequences(sequences, maxlen=100)

In [25]:
import numpy as np

emotion_dict = {}
labels = df['sentiment']

for k in range(len(labels.unique())):
    emotion_dict[labels.unique()[k]] = k
    
encoded_labels = np.array(list(labels.map(lambda x: emotion_dict[x])))

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(sequences, encoded_labels, stratify=encoded_labels)

In [27]:
import tensorflow as tf

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1024).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((X_test,y_test)).shuffle(1024).batch(32)

In [48]:
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout

model = tf.keras.Sequential()

model.add(Embedding(len(tokenizer.word_index)+1, 50))
model.add(LSTM(30, dropout=0.4))
model.add(Dense(16, activation='relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(7, activation='softmax'))

model.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 50)          453200    
_________________________________________________________________
lstm_5 (LSTM)                (None, 30)                9720      
_________________________________________________________________
dense_10 (Dense)             (None, 16)                496       
_________________________________________________________________
dropout_5 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 7)                 119       
Total params: 463,535
Trainable params: 463,535
Non-trainable params: 0
_________________________________________________________________


In [49]:
import tensorflow as tf
assert tf.__version__ >= "2.0"

In [50]:
%load_ext tensorboard
!rm -rf ./tensorboard-logs/ # Clear any logs from previous runs

import datetime 
import os

date = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = os.path.join("./tensorboard-logs/", date)
print("Writing logs to", log_dir)
exp_dir = os.path.join(log_dir, "softmax")

tb_callback = tf.keras.callbacks.TensorBoard(log_dir=exp_dir)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Writing logs to ./tensorboard-logs/20191030-180708


In [51]:
model.fit(X_train, y_train, 
          epochs = 10, 
          validation_data=(X_test,y_test),
         callbacks=[tb_callback])

Train on 5749 samples, validate on 1917 samples
Epoch 1/10
5749/5749 [==============================] - 11s 2ms/sample - loss: 1.9145 - accuracy: 0.1726 - val_loss: 1.8166 - val_accuracy: 0.2467
Epoch 2/10
5749/5749 [==============================] - 8s 1ms/sample - loss: 1.6899 - accuracy: 0.2961 - val_loss: 1.6041 - val_accuracy: 0.3459
Epoch 3/10
5749/5749 [==============================] - 8s 1ms/sample - loss: 1.4368 - accuracy: 0.3992 - val_loss: 1.5486 - val_accuracy: 0.3907
Epoch 4/10
5749/5749 [==============================] - 8s 1ms/sample - loss: 1.2425 - accuracy: 0.4818 - val_loss: 1.5215 - val_accuracy: 0.4231
Epoch 5/10
5749/5749 [==============================] - 8s 1ms/sample - loss: 1.0927 - accuracy: 0.5490 - val_loss: 1.5510 - val_accuracy: 0.4293
Epoch 6/10
5749/5749 [==============================] - 8s 1ms/sample - loss: 1.0309 - accuracy: 0.5733 - val_loss: 1.6260 - val_accuracy: 0.4319
Epoch 7/10
5749/5749 [==============================] - 8s 1ms/sample - los

In [52]:
%tensorboard --logdir "$log_dir"